In [1]:
import numpy as np
import pandas as pd
from IPython.display import display

a = 0.2
b = 1.2
h = 0.1
y0 = 0.25
N = int((b-a)/h + 1)
x = np.linspace(a, b, N)
y = np.linspace(y0, y0, N)
eps = 10**-6


def f(x : float, y : float):
    return 0.173*(x**2 + np.cos(0.7*x)) + 0.754*y

def f_u(x : float, y : float):
    return 0.754

def calculate_norm(y_prev, y_next):
    norm = -np.inf
    for i in range(len(y_prev)):
        norm = max(np.abs(y_prev[i] - y_next[2*i]), norm)
    return norm

def euler_iteration(x, y, N):
    h = x[1] - x[0]
    for i in range(0, N-1):
        y[i+1] = y[i] + h * f(x[i], y[i])
    return y

def newton_iteration(x, y, N, eps_newton=10**-8):
    h = x[1] - x[0]
    for i in range(0, N-1):
        y_k = y[i]
        y_k1 = y_k - (y_k - y[i] - 0.5 * h * (f(x[i], y[i]) + f(x[i+1], y_k)))/(1 - 0.5 * h * f_u(x[i+1], y_k))
        while (np.abs(y_k1 - y_k) >= eps_newton):
            y_k = y_k1
            y_k1 = y_k - (y_k - y[i] - 0.5 * h * (f(x[i], y[i]) + f(x[i+1], y_k)))/(1 - 0.5 * h * f_u(x[i+1], y_k))
        y[i+1] = y_k1
    return y

def runge_kutte_iteration(x, y, N):
    h = x[1] - x[0]
    for i in range(0, N-1):
        phi_0 = f(x[i], y[i])
        phi_1 = f(x[i] + h/2, y[i] + 0.5*h*phi_0)
        phi_2 = f(x[i] + h/2, y[i] + 0.5*h*phi_1)
        phi_3 = f(x[i+1], y[i] + h * phi_2)
        y[i+1] = y[i] + h*(phi_0 + 2*phi_1 + 2*phi_2 + phi_3)/6
    return y

def euler(N_start = 2, eps = 10**-8):
    N_prev = N_start
    while True:
        N_next = N_prev * 2
        x_prev = np.linspace(a, b, N_prev)
        x_next = np.linspace(a, b, N_next)
        y_prev = np.linspace(y0, y0, N_prev)
        y_next = np.linspace(y0, y0, N_next)

        y_prev = euler_iteration(x_prev, y_prev, N_prev)
        y_next = euler_iteration(x_next, y_next, N_next)

        norm = calculate_norm(y_prev, y_next)
        if (norm < eps):
            break
        N_prev = N_prev * 2   

    euler_df = pd.DataFrame(columns=["x", "y"])
    euler_df.x = x_next
    euler_df.y = y_next
    return euler_df

def trapezii(N_start = 2, eps = 10**-8, eps_newton = 10**-10):
    N_prev = N_start
    while True:
        N_next = N_prev * 2
        x_prev = np.linspace(a, b, N_prev)
        x_next = np.linspace(a, b, N_next)
        y_prev = np.linspace(y0, y0, N_prev)
        y_next = np.linspace(y0, y0, N_next)

        y_prev = newton_iteration(x_prev, y_prev, N_prev, eps_newton=eps_newton)
        y_next = newton_iteration(x_next, y_next, N_next, eps_newton=eps_newton)

        norm = calculate_norm(y_prev, y_next)
        if (norm/3 < eps):
            break
        N_prev = N_prev * 2

    trapezii_df = pd.DataFrame(columns=["x", "y"])
    trapezii_df.x = x_next
    trapezii_df.y = y_next
    return trapezii_df

def runge_kutte(N_start = 2, eps = 10**-8):
    N_prev = N_start
    while True:
        N_next = N_prev * 2
        x_prev = np.linspace(a, b, N_prev)
        x_next = np.linspace(a, b, N_next)
        y_prev = np.linspace(y0, y0, N_prev)
        y_next = np.linspace(y0, y0, N_next)

        y_prev = runge_kutte_iteration(x_prev, y_prev, N_prev)
        y_next = runge_kutte_iteration(x_next, y_next, N_next)

        norm = calculate_norm(y_prev, y_next)
        if (norm/15 < eps):
            break
        N_prev = N_prev * 2

    runge_kutte_df = pd.DataFrame(columns=["x", "y"])
    runge_kutte_df.x = x_next
    runge_kutte_df.y = y_next
    return runge_kutte_df

In [2]:
euler_df = euler(eps=10**-6)
print('euler:')
display(euler_df)

euler:


,x,y
0,0.200000,0.250000
1,0.200001,0.250000
2,0.200002,0.250001
3,0.200003,0.250001
4,0.200004,0.250001
...,...,...
1048571,1.199996,0.885690
1048572,1.199997,0.885691
1048573,1.199998,0.885692
1048574,1.199999,0.885693


In [3]:
trapezii_df = trapezii(eps=10**-6, eps_newton=10**-8)
print('trapezii:')
display(trapezii_df)

trapezii:


,x,y
0,0.200000,0.250000
1,0.200002,0.250001
2,0.200004,0.250001
3,0.200006,0.250002
4,0.200008,0.250003
...,...,...
524283,1.199992,0.885686
524284,1.199994,0.885688
524285,1.199996,0.885690
524286,1.199998,0.885692


In [4]:
runge_kutte_df = runge_kutte(eps=10**-6)
print('runge kutte:')
display(runge_kutte_df)

runge kutte:


,x,y
0,0.200000,0.250000
1,0.200008,0.250003
2,0.200015,0.250006
3,0.200023,0.250008
4,0.200031,0.250011
...,...,...
131067,1.199969,0.885663
131068,1.199977,0.885671
131069,1.199985,0.885679
131070,1.199992,0.885686


In [34]:
euler_df[(euler_df.x % 0.1 <= 0.000001) | (euler_df.x % 0.1 >= 0.099999)]

,x,y
0,0.200000,0.250000
1,0.200001,0.250000
104857,0.300000,0.288403
104858,0.300000,0.288404
209714,0.399999,0.330628
209715,0.400000,0.330629
209716,0.400001,0.330629
314572,0.500000,0.377249
314573,0.500000,0.377250
419429,0.599999,0.428885


In [35]:
trapezii_df[(trapezii_df.x % 0.1 <= 0.000001) | (trapezii_df.x % 0.1 >= 0.099999)]

,x,y
0,0.200000,0.250000
52429,0.300001,0.288404
104857,0.399999,0.330628
157286,0.500000,0.377249
209715,0.600000,0.428885
262143,0.699999,0.486206
262144,0.700001,0.486207
314572,0.800000,0.549939
367001,0.900000,0.620869
419430,1.000001,0.699845


In [37]:
runge_kutte_df[(runge_kutte_df.x % 0.1 <= 0.00001) | (runge_kutte_df.x % 0.1 >= 0.09999)]

,x,y
0,0.200000,0.250000
8192,0.300001,0.288404
16383,0.399990,0.330625
16384,0.400002,0.330630
24575,0.499991,0.377245
24576,0.500004,0.377251
32767,0.599993,0.428881
32768,0.600005,0.428888
40959,0.699994,0.486203
40960,0.700006,0.486210
